In [13]:
# ============================================================
# Q1-READY HISTOLOGY FEATURE EXTRACTION WITH FULL PREPROCESSING
# Includes: QC, Artifact Detection, Advanced Segmentation
# ============================================================

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import numpy as np
import pandas as pd
import openslide
import torch
import torchvision.transforms as transforms
from PIL import Image
from skimage.filters import threshold_otsu, laplace
from skimage.morphology import remove_small_objects, binary_dilation, disk
from skimage.color import rgb2hsv, rgb2gray
from skimage.measure import regionprops, label
from scipy.ndimage import gaussian_filter
import cv2
import timm
import json
from datetime import datetime
import warnings
import traceback
warnings.filterwarnings("ignore")

In [14]:


# ===============================
# CONFIGURATION - Q1 STANDARD
# ===============================

SVS_DIR = r"C:\Users\Shahinur\Downloads\PKG_Dataset\PKG - Brain-Mets-Lung-MRI-Path-Segs_histopathology images\data"
OUTPUT_CSV = "HISTOLOGY_FEATURES_Q1.csv"
QC_REPORT_CSV = "SLIDE_QUALITY_REPORT.csv"
CHECKPOINT_FILE = "checkpoint_q1.json"
PROGRESS_LOG = "progress_q1.txt"


In [15]:


# ---- Tiling Parameters ----
TILE_SIZE = 224
TILE_OVERLAP = 0  # Set to 112 for 50% overlap if needed
DOWNSAMPLE_TARGET = 1  # 40× magnification
NUM_CNN_TILES = 100  # Increased for better coverage
MIN_TISSUE_PERCENT = 0.60  # Stricter threshold

# ---- Quality Control Thresholds ----
MIN_BLUR_THRESHOLD = 100  # Laplacian variance
MIN_TILES_PER_SLIDE = 10
MAX_WHITESPACE_PERCENT = 0.90
MIN_SATURATION = 0.05  # HSV saturation for stain quality

# ---- Processing ----
SAVE_FREQUENCY = 3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("="*70)
print("Q1-READY HISTOLOGY FEATURE EXTRACTION PIPELINE")
print("="*70)
print(f"🔧 Device: {DEVICE}")
print(f"🔬 Model: DINO-ViT (vit_small_patch16_224)")
print(f"📊 Magnification: ~40× (downsample={DOWNSAMPLE_TARGET})")
print(f"🎯 Tiles/slide: {NUM_CNN_TILES}")
print(f"✅ Quality Control: ENABLED")
print(f"📁 Input: {SVS_DIR}\n")


Q1-READY HISTOLOGY FEATURE EXTRACTION PIPELINE
🔧 Device: cpu
🔬 Model: DINO-ViT (vit_small_patch16_224)
📊 Magnification: ~40× (downsample=1)
🎯 Tiles/slide: 100
✅ Quality Control: ENABLED
📁 Input: C:\Users\Shahinur\Downloads\PKG_Dataset\PKG - Brain-Mets-Lung-MRI-Path-Segs_histopathology images\data



In [16]:

# ===============================
# CHECKPOINTING
# ===============================

def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        try:
            with open(CHECKPOINT_FILE, "r") as f:
                cp = json.load(f)
                print(f"📋 Checkpoint loaded: {len(cp['processed'])} slides completed")
                return cp
        except:
            pass
    return {"processed": [], "failed": []}

def save_checkpoint(processed, failed):
    try:
        with open(CHECKPOINT_FILE, "w") as f:
            json.dump({
                "processed": processed,
                "failed": failed,
                "time": datetime.now().isoformat()
            }, f, indent=2)
    except Exception as e:
        print(f"⚠️ Checkpoint save failed: {e}")

def log_progress(msg):
    try:
        with open(PROGRESS_LOG, "a") as f:
            f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")
    except:
        pass


In [17]:

# ===============================
# QUALITY CONTROL FUNCTIONS
# ===============================

def assess_blur(tile_rgb):
    """Detect blurry/out-of-focus tiles using Laplacian variance"""
    gray = rgb2gray(tile_rgb)
    laplacian_var = laplace(gray).var()
    return laplacian_var

def detect_pen_marks(tile_rgb):
    """Detect pen/marker artifacts using color analysis"""
    hsv = rgb2hsv(tile_rgb)
    
    # Pen marks are typically highly saturated
    high_sat_mask = hsv[:, :, 1] > 0.7
    high_sat_ratio = high_sat_mask.sum() / high_sat_mask.size
    
    # Check for extreme colors (blue/green/black pens)
    blue_mask = (tile_rgb[:, :, 2] > 150) & (tile_rgb[:, :, 0] < 100)
    green_mask = (tile_rgb[:, :, 1] > 150) & (tile_rgb[:, :, 0] < 100)
    
    artifact_ratio = (blue_mask.sum() + green_mask.sum()) / blue_mask.size
    
    return high_sat_ratio > 0.15 or artifact_ratio > 0.10

def assess_stain_quality(tile_rgb):
    """Check H&E stain quality using HSV saturation"""
    hsv = rgb2hsv(tile_rgb)
    mean_saturation = np.mean(hsv[:, :, 1])
    return mean_saturation

def is_mostly_background(tile_rgb, threshold=220):
    """Check if tile is mostly white/background"""
    mean_intensity = np.mean(tile_rgb)
    return mean_intensity > threshold

In [18]:


# ===============================
# ADVANCED TISSUE SEGMENTATION
# ===============================

def advanced_tissue_mask(rgb):
    """
    Advanced tissue segmentation with artifact removal
    Better than simple Otsu thresholding
    """
    # Convert to grayscale
    gray = np.mean(rgb, axis=2)
    
    # Initial Otsu thresholding
    try:
        thresh = threshold_otsu(gray)
    except:
        thresh = 200
    
    mask = gray < thresh
    
    # Remove small objects (debris)
    mask = remove_small_objects(mask, min_size=500)
    
    # Fill holes in tissue
    mask = binary_dilation(mask, disk(3))
    
    # Additional filtering: remove pen marks
    hsv = rgb2hsv(rgb)
    saturated = hsv[:, :, 1] > 0.7
    
    # Remove highly saturated regions (likely artifacts)
    artifact_mask = binary_dilation(saturated, disk(5))
    mask = mask & ~artifact_mask
    
    return mask


In [19]:

# ===============================
# STAIN NORMALIZATION (ENHANCED REINHARD)
# ===============================

def reinhard_normalization(img,
                           target_means=(0.75, 0.55, 0.45),
                           target_stds=(0.15, 0.15, 0.15)):
    """
    Reinhard color normalization for H&E slides
    Reference: Reinhard et al., 2001
    """
    img = img.astype(np.float32) / 255.0
    
    # Compute source statistics
    means = img.mean(axis=(0, 1))
    stds = img.std(axis=(0, 1)) + 1e-8
    
    # Normalize
    norm = (img - means) / stds
    norm = norm * target_stds + target_means
    norm = np.clip(norm, 0, 1)
    
    return (norm * 255).astype(np.uint8)


In [20]:

# ===============================
# TILE EXTRACTION WITH QC
# ===============================

def extract_qc_tiles(slide, max_tiles=NUM_CNN_TILES):
    """
    Extract high-quality tissue tiles with comprehensive QC
    """
    try:
        level = slide.get_best_level_for_downsample(DOWNSAMPLE_TARGET)
        downsample = slide.level_downsamples[level]
        level_width, level_height = slide.level_dimensions[level]
        
        print(f"  📐 Level {level}, Downsample: {downsample:.2f}×")
        print(f"  📏 Dimensions: {level_width}×{level_height}")
        
        tiles = []
        qc_stats = {
            'total_sampled': 0,
            'passed_tissue': 0,
            'failed_blur': 0,
            'failed_artifacts': 0,
            'failed_background': 0,
            'failed_stain': 0
        }
        
        # Grid sampling with optional overlap
        step = TILE_SIZE - TILE_OVERLAP
        
        for y in range(0, level_height - TILE_SIZE, step):
            for x in range(0, level_width - TILE_SIZE, step):
                
                if len(tiles) >= max_tiles:
                    break
                
                qc_stats['total_sampled'] += 1
                
                # Convert to level 0 coordinates
                x0 = int(x * downsample)
                y0 = int(y * downsample)
                
                # Read tile
                tile = slide.read_region(
                    (x0, y0),
                    level,
                    (TILE_SIZE, TILE_SIZE)
                ).convert("RGB")
                
                tile_np = np.array(tile)
                
                # QC Step 1: Check for background
                if is_mostly_background(tile_np):
                    qc_stats['failed_background'] += 1
                    continue
                
                # QC Step 2: Tissue segmentation
                mask = advanced_tissue_mask(tile_np)
                tissue_percent = mask.sum() / mask.size
                
                if tissue_percent < MIN_TISSUE_PERCENT:
                    continue
                
                qc_stats['passed_tissue'] += 1
                
                # QC Step 3: Blur detection
                blur_score = assess_blur(tile_np)
                if blur_score < MIN_BLUR_THRESHOLD:
                    qc_stats['failed_blur'] += 1
                    continue
                
                # QC Step 4: Artifact detection
                if detect_pen_marks(tile_np):
                    qc_stats['failed_artifacts'] += 1
                    continue
                
                # QC Step 5: Stain quality
                stain_quality = assess_stain_quality(tile_np)
                if stain_quality < MIN_SATURATION:
                    qc_stats['failed_stain'] += 1
                    continue
                
                # Apply stain normalization
                tile_np = reinhard_normalization(tile_np)
                
                tiles.append(tile_np)
            
            if len(tiles) >= max_tiles:
                break
        
        # Print QC summary
        print(f"  ✅ QC Summary:")
        print(f"     Sampled: {qc_stats['total_sampled']}")
        print(f"     Passed tissue: {qc_stats['passed_tissue']}")
        print(f"     Failed blur: {qc_stats['failed_blur']}")
        print(f"     Failed artifacts: {qc_stats['failed_artifacts']}")
        print(f"     Failed stain: {qc_stats['failed_stain']}")
        print(f"     Final tiles: {len(tiles)}")
        
        return tiles, qc_stats
        
    except Exception as e:
        print(f"  ✗ Tile extraction error: {e}")
        return [], {}


In [21]:

# ===============================
# SLIDE METADATA EXTRACTION
# ===============================

def extract_slide_metadata(slide):
    """Extract comprehensive slide metadata for reproducibility"""
    metadata = {}
    
    try:
        props = slide.properties
        
        # Basic properties
        metadata['width'] = slide.dimensions[0]
        metadata['height'] = slide.dimensions[1]
        metadata['num_levels'] = len(slide.level_dimensions)
        
        # Vendor info
        metadata['vendor'] = props.get('openslide.vendor', 'unknown')
        metadata['scanner'] = props.get('aperio.ScanScope ID', 'unknown')
        
        # Magnification
        metadata['objective_power'] = props.get('openslide.objective-power', 'unknown')
        metadata['mpp_x'] = props.get('openslide.mpp-x', 'unknown')
        metadata['mpp_y'] = props.get('openslide.mpp-y', 'unknown')
        
        # Staining info (if available)
        metadata['stain'] = props.get('aperio.Stain', 'H&E')
        
    except Exception as e:
        print(f"  ⚠️ Metadata extraction warning: {e}")
    
    return metadata

In [22]:


# ===============================
# DINO-ViT FEATURE EXTRACTOR
# ===============================

class DinoViTExtractor:
    """DINO-ViT feature extractor with robust aggregation"""
    
    def __init__(self):
        print("🔄 Loading DINO-ViT model...")
        try:
            self.model = timm.create_model(
                "vit_small_patch16_224.dino",
                pretrained=True,
                num_classes=0
            ).to(DEVICE).eval()
            print("✅ DINO-ViT loaded successfully\n")
        except Exception as e:
            print(f"✗ Model loading failed: {e}")
            raise

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

    def extract(self, tiles):
        """Extract features with outlier detection"""
        if len(tiles) == 0:
            return None, None
        
        feats = []
        
        print(f"  🔬 Extracting DINO features from {len(tiles)} tiles...")
        
        for i, tile in enumerate(tiles):
            try:
                img = Image.fromarray(tile)
                tensor = self.transform(img).unsqueeze(0).to(DEVICE)

                with torch.no_grad():
                    f = self.model(tensor).squeeze().cpu().numpy()
                    feats.append(f)
                
                if (i + 1) % 20 == 0:
                    print(f"     {i+1}/{len(tiles)}", end='\r')
                    
            except Exception as e:
                print(f"  ⚠️ Tile {i} failed: {e}")
                continue
        
        if len(feats) == 0:
            return None, None
        
        feats = np.array(feats)
        print(f"  ✅ Feature shape: {feats.shape}")
        
        # Outlier detection using z-score
        z_scores = np.abs((feats - feats.mean(axis=0)) / (feats.std(axis=0) + 1e-8))
        outlier_mask = (z_scores > 3).any(axis=1)
        num_outliers = outlier_mask.sum()
        
        if num_outliers > 0:
            print(f"  🔍 Removed {num_outliers} outlier tiles")
            feats = feats[~outlier_mask]
        
        # Comprehensive aggregation
        agg_features = {
            "dino_mean": feats.mean(axis=0),
            "dino_std": feats.std(axis=0),
            "dino_max": feats.max(axis=0),
            "dino_min": feats.min(axis=0),
            "dino_median": np.median(feats, axis=0),
            "dino_q25": np.percentile(feats, 25, axis=0),
            "dino_q75": np.percentile(feats, 75, axis=0),
        }
        
        # Feature statistics for QC
        feature_stats = {
            'num_tiles_used': len(feats),
            'num_outliers_removed': num_outliers,
            'feature_dim': feats.shape[1],
            'mean_norm': float(np.linalg.norm(feats.mean(axis=0))),
            'std_norm': float(np.linalg.norm(feats.std(axis=0)))
        }
        
        return agg_features, feature_stats


In [23]:

# ===============================
# SLIDE PROCESSING PIPELINE
# ===============================

def process_slide_q1(path, extractor):
    """Q1-standard slide processing with full QC"""
    
    qc_report = {
        'slide_id': os.path.basename(path),
        'status': 'failed',
        'failure_reason': None
    }
    
    try:
        # Open slide
        slide = openslide.OpenSlide(path)
        
        # Extract metadata
        metadata = extract_slide_metadata(slide)
        print(f"  📋 Vendor: {metadata.get('vendor', 'unknown')}")
        print(f"  📏 Size: {metadata['width']}×{metadata['height']}")
        
        # Extract tiles with QC
        tiles, qc_stats = extract_qc_tiles(slide, max_tiles=NUM_CNN_TILES)
        slide.close()
        
        # Check minimum tile requirement
        if len(tiles) < MIN_TILES_PER_SLIDE:
            qc_report['failure_reason'] = f"Insufficient tiles: {len(tiles)} < {MIN_TILES_PER_SLIDE}"
            qc_report['num_tiles'] = len(tiles)
            return None, qc_report
        
        # Extract features
        features, feature_stats = extractor.extract(tiles)
        
        if features is None:
            qc_report['failure_reason'] = "Feature extraction failed"
            return None, qc_report
        
        # Flatten features
        flat = {}
        for k, v in features.items():
            for i, val in enumerate(v):
                flat[f"{k}_{i}"] = float(val)
        
        # Add metadata
        flat.update({
            'slide_id': os.path.basename(path),
            'num_tiles': len(tiles),
            'slide_width': metadata['width'],
            'slide_height': metadata['height'],
            'vendor': metadata['vendor'],
            'objective_power': metadata['objective_power']
        })
        
        # Update QC report
        qc_report.update({
            'status': 'success',
            'num_tiles': len(tiles),
            'qc_sampled': qc_stats.get('total_sampled', 0),
            'qc_passed': qc_stats.get('passed_tissue', 0),
            'qc_blur_failed': qc_stats.get('failed_blur', 0),
            'qc_artifact_failed': qc_stats.get('failed_artifacts', 0),
            'feature_dim': feature_stats['feature_dim'],
            'outliers_removed': feature_stats['num_outliers_removed']
        })
        
        return flat, qc_report
        
    except Exception as e:
        qc_report['failure_reason'] = str(e)
        print(f"  ✗ Processing error: {e}")
        traceback.print_exc()
        return None, qc_report

In [24]:


# ===============================
# MAIN PIPELINE
# ===============================

def main():
    print("\n" + "="*70)
    
    # Initialize
    extractor = DinoViTExtractor()
    checkpoint = load_checkpoint()
    processed = set(checkpoint["processed"])
    failed = set(checkpoint.get("failed", []))
    
    # Get files
    all_files = [f for f in os.listdir(SVS_DIR) if f.lower().endswith(".svs")]
    remaining = [f for f in all_files if f not in processed and f not in failed]
    
    print(f"\n📊 Pipeline Status:")
    print(f"   Total slides: {len(all_files)}")
    print(f"   Completed: {len(processed)}")
    print(f"   Failed: {len(failed)}")
    print(f"   Remaining: {len(remaining)}\n")
    
    if len(remaining) == 0:
        print("✅ All slides processed!\n")
        return
    
    results = []
    qc_reports = []
    newly_failed = []
    
    for idx, fname in enumerate(remaining, 1):
        print(f"\n{'='*70}")
        print(f"[{idx}/{len(remaining)}] Processing: {fname}")
        print(f"{'='*70}")
        log_progress(f"Starting: {fname}")
        
        path = os.path.join(SVS_DIR, fname)
        
        feats, qc_report = process_slide_q1(path, extractor)
        qc_reports.append(qc_report)
        
        if feats is None:
            print(f"  ❌ FAILED: {qc_report['failure_reason']}")
            log_progress(f"Failed: {fname} - {qc_report['failure_reason']}")
            newly_failed.append(fname)
            continue
        
        results.append(feats)
        processed.add(fname)
        
        print(f"  ✅ SUCCESS")
        log_progress(f"Completed: {fname}")
        
        # Periodic save
        if len(results) % SAVE_FREQUENCY == 0:
            print(f"\n💾 Checkpoint save ({len(results)} slides)...")
            pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
            pd.DataFrame(qc_reports).to_csv(QC_REPORT_CSV, index=False)
            save_checkpoint(list(processed), list(failed) + newly_failed)
    
    # Final save
    if len(results) > 0:
        print(f"\n{'='*70}")
        print("💾 FINAL SAVE")
        print(f"{'='*70}")
        pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
        pd.DataFrame(qc_reports).to_csv(QC_REPORT_CSV, index=False)
        save_checkpoint(list(processed), list(failed) + newly_failed)
        
        print(f"✅ Features saved: {OUTPUT_CSV}")
        print(f"✅ QC report saved: {QC_REPORT_CSV}")
        print(f"✅ Processed: {len(results)} slides")
        print(f"❌ Failed: {len(newly_failed)} slides\n")
    
    print("="*70)
    print("PIPELINE COMPLETED")
    print("="*70 + "\n")


In [25]:


if __name__ == "__main__":
    main()


🔄 Loading DINO-ViT model...
✅ DINO-ViT loaded successfully


📊 Pipeline Status:
   Total slides: 111
   Completed: 0
   Failed: 0
   Remaining: 111


[1/111] Processing: YG_0CBM148C1MFN_wsi.svs
  📋 Vendor: aperio
  📏 Size: 22167×51785
  📐 Level 0, Downsample: 1.00×
  📏 Dimensions: 22167×51785
  ✅ QC Summary:
     Sampled: 22638
     Passed tissue: 408
     Failed blur: 408
     Failed artifacts: 0
     Failed stain: 0
     Final tiles: 0
  ❌ FAILED: Insufficient tiles: 0 < 10

[2/111] Processing: YG_0PGQQ6USQ9JB_wsi.svs
  📋 Vendor: aperio
  📏 Size: 42011×48050
  📐 Level 0, Downsample: 1.00×
  📏 Dimensions: 42011×48050


KeyboardInterrupt: 

In [29]:
# ============================================================
# Q1-READY HISTOLOGY FEATURE EXTRACTION WITH ADAPTIVE QC
# Includes: Adaptive Blur Threshold, Full Preprocessing
# ============================================================

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import numpy as np
import pandas as pd
import openslide
import torch
import torchvision.transforms as transforms
from PIL import Image
from skimage.filters import threshold_otsu, laplace
from skimage.morphology import remove_small_objects, binary_dilation, disk
from skimage.color import rgb2hsv, rgb2gray
import timm
import json
from datetime import datetime
import warnings
import traceback
warnings.filterwarnings("ignore")

# ===============================
# CONFIGURATION - Q1 STANDARD WITH ADAPTIVE QC
# ===============================

SVS_DIR = r"C:\Users\Shahinur\Downloads\PKG_Dataset\PKG - Brain-Mets-Lung-MRI-Path-Segs_histopathology images\data"
OUTPUT_CSV = "HISTOLOGY_FEATURES_Q1_ADAPTIVE.csv"
QC_REPORT_CSV = "SLIDE_QUALITY_REPORT_ADAPTIVE.csv"
QC_STATISTICS_JSON = "qc_statistics_adaptive.json"
CHECKPOINT_FILE = "checkpoint_q1_adaptive.json"
PROGRESS_LOG = "progress_q1_adaptive.txt"

# ---- Tiling Parameters ----
TILE_SIZE = 224
TILE_OVERLAP = 0
DOWNSAMPLE_TARGET = 1
NUM_CNN_TILES = 100
MIN_TISSUE_PERCENT = 0.40  # REDUCED: More lenient tissue requirement (was 0.60)

# ---- Quality Control - ADAPTIVE (OPTIMIZED FOR THIS DATASET) ----
USE_ADAPTIVE_BLUR = False  # DISABLED: Dataset has uniform low contrast
BLUR_PERCENTILE = 5
NUM_CALIBRATION_TILES = 500
FALLBACK_BLUR_THRESHOLD = 0.1  # VERY LOW: Based on observed data (was 20.0)

MIN_TILES_PER_SLIDE = 10
MAX_WHITESPACE_PERCENT = 0.90
MIN_SATURATION = 0.02  # Lenient for low-contrast staining

# ---- Processing ----
SAVE_FREQUENCY = 3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("="*70)
print("Q1-READY HISTOLOGY PIPELINE - OPTIMIZED FOR LOW-CONTRAST DATA")
print("="*70)
print(f"🔧 Device: {DEVICE}")
print(f"🔬 Model: DINO-ViT")
print(f"📊 Adaptive Blur: {'ENABLED' if USE_ADAPTIVE_BLUR else 'DISABLED (uniform quality)'}")
print(f"🎯 Blur Percentile: {BLUR_PERCENTILE}th")
print(f"📁 Input: {SVS_DIR}\n")

# ===============================
# GLOBAL QC STATISTICS
# ===============================

GLOBAL_QC_STATS = {
    'blur_scores': [],
    'saturation_scores': [],
    'tissue_percentages': [],
    'adaptive_blur_threshold': None,
    'calibration_complete': False
}

# ===============================
# CHECKPOINTING
# ===============================

def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        try:
            with open(CHECKPOINT_FILE, "r") as f:
                cp = json.load(f)
                print(f"📋 Checkpoint: {len(cp['processed'])} slides completed")
                return cp
        except:
            pass
    return {"processed": [], "failed": []}

def save_checkpoint(processed, failed):
    try:
        with open(CHECKPOINT_FILE, "w") as f:
            json.dump({
                "processed": processed,
                "failed": failed,
                "time": datetime.now().isoformat(),
                "adaptive_threshold": GLOBAL_QC_STATS.get('adaptive_blur_threshold')
            }, f, indent=2)
    except Exception as e:
        print(f"⚠️ Checkpoint save failed: {e}")

def log_progress(msg):
    try:
        with open(PROGRESS_LOG, "a") as f:
            f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")
    except:
        pass

def save_qc_statistics():
    """Save global QC statistics for reproducibility"""
    try:
        stats = {
            'blur_threshold': GLOBAL_QC_STATS['adaptive_blur_threshold'],
            'blur_percentile': BLUR_PERCENTILE,
            'num_calibration_tiles': len(GLOBAL_QC_STATS['blur_scores']),
            'blur_stats': {
                'mean': float(np.mean(GLOBAL_QC_STATS['blur_scores'])),
                'std': float(np.std(GLOBAL_QC_STATS['blur_scores'])),
                'min': float(np.min(GLOBAL_QC_STATS['blur_scores'])),
                'max': float(np.max(GLOBAL_QC_STATS['blur_scores'])),
                'p10': float(np.percentile(GLOBAL_QC_STATS['blur_scores'], 10)),
                'p50': float(np.percentile(GLOBAL_QC_STATS['blur_scores'], 50)),
                'p90': float(np.percentile(GLOBAL_QC_STATS['blur_scores'], 90))
            },
            'saturation_stats': {
                'mean': float(np.mean(GLOBAL_QC_STATS['saturation_scores'])),
                'std': float(np.std(GLOBAL_QC_STATS['saturation_scores']))
            } if GLOBAL_QC_STATS['saturation_scores'] else None,
            'timestamp': datetime.now().isoformat()
        }
        
        with open(QC_STATISTICS_JSON, 'w') as f:
            json.dump(stats, f, indent=2)
        
        print(f"📊 QC statistics saved: {QC_STATISTICS_JSON}")
        
    except Exception as e:
        print(f"⚠️ Could not save QC stats: {e}")

# ===============================
# QUALITY CONTROL FUNCTIONS
# ===============================

def assess_blur(tile_rgb):
    """Detect blur using Laplacian variance - optimized for histology"""
    gray = rgb2gray(tile_rgb)
    
    # Use both Laplacian variance and gradient magnitude
    laplacian_var = laplace(gray).var()
    
    # For very low variance images, check gradient too
    if laplacian_var < 10:
        gradient = np.gradient(gray)
        grad_mag = np.sqrt(gradient[0]**2 + gradient[1]**2).mean()
        return laplacian_var + grad_mag * 10  # Boost score
    
    return laplacian_var

def detect_pen_marks(tile_rgb):
    """Detect pen/marker artifacts"""
    hsv = rgb2hsv(tile_rgb)
    
    high_sat_mask = hsv[:, :, 1] > 0.7
    high_sat_ratio = high_sat_mask.sum() / high_sat_mask.size
    
    blue_mask = (tile_rgb[:, :, 2] > 150) & (tile_rgb[:, :, 0] < 100)
    green_mask = (tile_rgb[:, :, 1] > 150) & (tile_rgb[:, :, 0] < 100)
    
    artifact_ratio = (blue_mask.sum() + green_mask.sum()) / blue_mask.size
    
    return high_sat_ratio > 0.15 or artifact_ratio > 0.10

def assess_stain_quality(tile_rgb):
    """Check H&E stain quality"""
    hsv = rgb2hsv(tile_rgb)
    mean_saturation = np.mean(hsv[:, :, 1])
    return mean_saturation

def is_mostly_background(tile_rgb, threshold=220):
    """Check if tile is mostly background"""
    mean_intensity = np.mean(tile_rgb)
    return mean_intensity > threshold

# ===============================
# ADVANCED TISSUE SEGMENTATION
# ===============================

def advanced_tissue_mask(rgb):
    """Advanced tissue segmentation with artifact removal"""
    gray = np.mean(rgb, axis=2)
    
    try:
        thresh = threshold_otsu(gray)
    except:
        thresh = 200
    
    mask = gray < thresh
    mask = remove_small_objects(mask, min_size=500)
    mask = binary_dilation(mask, disk(3))
    
    # Remove pen marks
    hsv = rgb2hsv(rgb)
    saturated = hsv[:, :, 1] > 0.7
    artifact_mask = binary_dilation(saturated, disk(5))
    mask = mask & ~artifact_mask
    
    return mask

# ===============================
# STAIN NORMALIZATION
# ===============================

def reinhard_normalization(img,
                           target_means=(0.75, 0.55, 0.45),
                           target_stds=(0.15, 0.15, 0.15)):
    """Reinhard color normalization"""
    img = img.astype(np.float32) / 255.0
    means = img.mean(axis=(0, 1))
    stds = img.std(axis=(0, 1)) + 1e-8
    norm = (img - means) / stds
    norm = norm * target_stds + target_means
    norm = np.clip(norm, 0, 1)
    return (norm * 255).astype(np.uint8)

# ===============================
# ADAPTIVE BLUR CALIBRATION
# ===============================

def calibrate_blur_threshold(slide_paths, num_tiles=NUM_CALIBRATION_TILES):
    """
    Calibrate adaptive blur threshold across dataset
    Samples tiles from multiple slides to determine threshold
    """
    print("\n" + "="*70)
    print("🔬 ADAPTIVE BLUR CALIBRATION")
    print("="*70)
    print(f"Sampling {num_tiles} tiles from {len(slide_paths)} slides...")
    
    blur_scores = []
    saturation_scores = []
    tissue_percentages = []
    
    tiles_per_slide = max(10, num_tiles // len(slide_paths))
    
    for idx, path in enumerate(slide_paths):
        try:
            print(f"  [{idx+1}/{len(slide_paths)}] Sampling: {os.path.basename(path)}", end='\r')
            
            slide = openslide.OpenSlide(path)
            level = slide.get_best_level_for_downsample(DOWNSAMPLE_TARGET)
            downsample = slide.level_downsamples[level]
            level_width, level_height = slide.level_dimensions[level]
            
            # Random sampling
            for _ in range(tiles_per_slide):
                if len(blur_scores) >= num_tiles:
                    break
                
                # Random position
                x = np.random.randint(0, max(1, level_width - TILE_SIZE))
                y = np.random.randint(0, max(1, level_height - TILE_SIZE))
                
                x0 = int(x * downsample)
                y0 = int(y * downsample)
                
                tile = slide.read_region(
                    (x0, y0), level, (TILE_SIZE, TILE_SIZE)
                ).convert("RGB")
                
                tile_np = np.array(tile)
                
                # Skip background
                if is_mostly_background(tile_np):
                    continue
                
                # Check tissue content
                mask = advanced_tissue_mask(tile_np)
                tissue_pct = mask.sum() / mask.size
                
                if tissue_pct < MIN_TISSUE_PERCENT:
                    continue
                
                # Calculate metrics
                blur_score = assess_blur(tile_np)
                sat_score = assess_stain_quality(tile_np)
                
                blur_scores.append(blur_score)
                saturation_scores.append(sat_score)
                tissue_percentages.append(tissue_pct)
            
            slide.close()
            
            if len(blur_scores) >= num_tiles:
                break
                
        except Exception as e:
            print(f"\n  ⚠️ Calibration error on {os.path.basename(path)}: {e}")
            continue
    
    print(f"\n✅ Collected {len(blur_scores)} calibration tiles")
    
    if len(blur_scores) < 50:
        print("⚠️ WARNING: Insufficient calibration data. Using fallback threshold.")
        print(f"   Fallback blur threshold: {FALLBACK_BLUR_THRESHOLD}")
        return FALLBACK_BLUR_THRESHOLD  # Use reduced fallback
    
    # Calculate adaptive threshold
    blur_array = np.array(blur_scores)
    adaptive_threshold = np.percentile(blur_array, BLUR_PERCENTILE)
    
    print(f"\n📊 Blur Statistics:")
    print(f"   Mean: {blur_array.mean():.2f}")
    print(f"   Std: {blur_array.std():.2f}")
    print(f"   Min: {blur_array.min():.2f}")
    print(f"   Max: {blur_array.max():.2f}")
    print(f"   P10: {np.percentile(blur_array, 10):.2f}")
    print(f"   P50: {np.percentile(blur_array, 50):.2f}")
    print(f"   P90: {np.percentile(blur_array, 90):.2f}")
    print(f"\n🎯 Adaptive Threshold ({BLUR_PERCENTILE}th percentile): {adaptive_threshold:.2f}")
    
    # Store for later reference
    GLOBAL_QC_STATS['blur_scores'] = blur_scores
    GLOBAL_QC_STATS['saturation_scores'] = saturation_scores
    GLOBAL_QC_STATS['tissue_percentages'] = tissue_percentages
    GLOBAL_QC_STATS['adaptive_blur_threshold'] = float(adaptive_threshold)
    GLOBAL_QC_STATS['calibration_complete'] = True
    
    save_qc_statistics()
    
    print("="*70 + "\n")
    
    return adaptive_threshold

# ===============================
# TILE EXTRACTION WITH ADAPTIVE QC
# ===============================

def extract_qc_tiles(slide, max_tiles=NUM_CNN_TILES, blur_threshold=None):
    """Extract tiles with adaptive QC"""
    
    if blur_threshold is None:
        blur_threshold = GLOBAL_QC_STATS.get('adaptive_blur_threshold', 50.0)
    
    try:
        level = slide.get_best_level_for_downsample(DOWNSAMPLE_TARGET)
        downsample = slide.level_downsamples[level]
        level_width, level_height = slide.level_dimensions[level]
        
        print(f"  📐 Level {level}, Downsample: {downsample:.2f}×")
        print(f"  📏 Dimensions: {level_width}×{level_height}")
        print(f"  🎯 Blur Threshold: {blur_threshold:.2f}")
        
        tiles = []
        qc_stats = {
            'total_sampled': 0,
            'passed_tissue': 0,
            'failed_blur': 0,
            'failed_artifacts': 0,
            'failed_background': 0,
            'failed_stain': 0,
            'blur_scores_passed': [],
            'blur_scores_failed': []
        }
        
        step = TILE_SIZE - TILE_OVERLAP
        
        for y in range(0, level_height - TILE_SIZE, step):
            for x in range(0, level_width - TILE_SIZE, step):
                
                if len(tiles) >= max_tiles:
                    break
                
                qc_stats['total_sampled'] += 1
                
                x0 = int(x * downsample)
                y0 = int(y * downsample)
                
                tile = slide.read_region(
                    (x0, y0), level, (TILE_SIZE, TILE_SIZE)
                ).convert("RGB")
                
                tile_np = np.array(tile)
                
                # QC Step 1: Background check
                if is_mostly_background(tile_np):
                    qc_stats['failed_background'] += 1
                    continue
                
                # QC Step 2: Tissue segmentation
                mask = advanced_tissue_mask(tile_np)
                tissue_percent = mask.sum() / mask.size
                
                if tissue_percent < MIN_TISSUE_PERCENT:
                    continue
                
                qc_stats['passed_tissue'] += 1
                
                # Calculate blur score BEFORE normalization (important!)
                blur_score = assess_blur(tile_np)
                
                # QC Step 3: ADAPTIVE Blur detection
                if USE_ADAPTIVE_BLUR and blur_score < blur_threshold:
                    qc_stats['failed_blur'] += 1
                    qc_stats['blur_scores_failed'].append(blur_score)
                    continue
                else:
                    qc_stats['blur_scores_passed'].append(blur_score)
                
                # QC Step 4: Artifact detection
                if detect_pen_marks(tile_np):
                    qc_stats['failed_artifacts'] += 1
                    continue
                
                # QC Step 5: Stain quality
                stain_quality = assess_stain_quality(tile_np)
                if stain_quality < MIN_SATURATION:
                    qc_stats['failed_stain'] += 1
                    continue
                
                # Apply normalization
                tile_np = reinhard_normalization(tile_np)
                tiles.append(tile_np)
            
            if len(tiles) >= max_tiles:
                break
        
        # Print QC summary
        print(f"  ✅ QC Summary:")
        print(f"     Sampled: {qc_stats['total_sampled']}")
        print(f"     Passed tissue: {qc_stats['passed_tissue']}")
        print(f"     Failed blur: {qc_stats['failed_blur']}")
        print(f"     Failed artifacts: {qc_stats['failed_artifacts']}")
        print(f"     Failed stain: {qc_stats['failed_stain']}")
        print(f"     Final tiles: {len(tiles)}")
        
        if qc_stats['blur_scores_passed']:
            avg_blur = np.mean(qc_stats['blur_scores_passed'])
            print(f"     Blur (passed): {avg_blur:.2f} ± {np.std(qc_stats['blur_scores_passed']):.2f}")
        if qc_stats['blur_scores_failed']:
            avg_blur_fail = np.mean(qc_stats['blur_scores_failed'])
            print(f"     Blur (failed): {avg_blur_fail:.2f} ± {np.std(qc_stats['blur_scores_failed']):.2f}")
            print(f"     ⚠️ Consider lowering blur threshold if too many failures")
        
        return tiles, qc_stats
        
    except Exception as e:
        print(f"  ✗ Tile extraction error: {e}")
        return [], {}

# ===============================
# SLIDE METADATA
# ===============================

def extract_slide_metadata(slide):
    """Extract slide metadata"""
    metadata = {}
    
    try:
        props = slide.properties
        metadata['width'] = slide.dimensions[0]
        metadata['height'] = slide.dimensions[1]
        metadata['num_levels'] = len(slide.level_dimensions)
        metadata['vendor'] = props.get('openslide.vendor', 'unknown')
        metadata['scanner'] = props.get('aperio.ScanScope ID', 'unknown')
        metadata['objective_power'] = props.get('openslide.objective-power', 'unknown')
        metadata['mpp_x'] = props.get('openslide.mpp-x', 'unknown')
        metadata['mpp_y'] = props.get('openslide.mpp-y', 'unknown')
        metadata['stain'] = props.get('aperio.Stain', 'H&E')
    except Exception as e:
        print(f"  ⚠️ Metadata extraction warning: {e}")
    
    return metadata

# ===============================
# DINO-ViT FEATURE EXTRACTOR
# ===============================

class DinoViTExtractor:
    """DINO-ViT feature extractor"""
    
    def __init__(self):
        print("🔄 Loading DINO-ViT model...")
        try:
            self.model = timm.create_model(
                "vit_small_patch16_224.dino",
                pretrained=True,
                num_classes=0
            ).to(DEVICE).eval()
            print("✅ DINO-ViT loaded successfully\n")
        except Exception as e:
            print(f"✗ Model loading failed: {e}")
            raise

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

    def extract(self, tiles):
        """Extract features with outlier detection"""
        if len(tiles) == 0:
            return None, None
        
        feats = []
        
        print(f"  🔬 Extracting DINO features from {len(tiles)} tiles...")
        
        for i, tile in enumerate(tiles):
            try:
                img = Image.fromarray(tile)
                tensor = self.transform(img).unsqueeze(0).to(DEVICE)

                with torch.no_grad():
                    f = self.model(tensor).squeeze().cpu().numpy()
                    feats.append(f)
                
                if (i + 1) % 20 == 0:
                    print(f"     {i+1}/{len(tiles)}", end='\r')
                    
            except Exception as e:
                print(f"  ⚠️ Tile {i} failed: {e}")
                continue
        
        if len(feats) == 0:
            return None, None
        
        feats = np.array(feats)
        print(f"  ✅ Feature shape: {feats.shape}")
        
        # Outlier detection
        z_scores = np.abs((feats - feats.mean(axis=0)) / (feats.std(axis=0) + 1e-8))
        outlier_mask = (z_scores > 3).any(axis=1)
        num_outliers = outlier_mask.sum()
        
        if num_outliers > 0:
            print(f"  🔍 Removed {num_outliers} outlier tiles")
            feats = feats[~outlier_mask]
        
        # Aggregation
        agg_features = {
            "dino_mean": feats.mean(axis=0),
            "dino_std": feats.std(axis=0),
            "dino_max": feats.max(axis=0),
            "dino_min": feats.min(axis=0),
            "dino_median": np.median(feats, axis=0),
            "dino_q25": np.percentile(feats, 25, axis=0),
            "dino_q75": np.percentile(feats, 75, axis=0),
        }
        
        feature_stats = {
            'num_tiles_used': len(feats),
            'num_outliers_removed': num_outliers,
            'feature_dim': feats.shape[1],
            'mean_norm': float(np.linalg.norm(feats.mean(axis=0))),
            'std_norm': float(np.linalg.norm(feats.std(axis=0)))
        }
        
        return agg_features, feature_stats

# ===============================
# SLIDE PROCESSING
# ===============================

def process_slide_q1(path, extractor, blur_threshold):
    """Q1-standard slide processing"""
    
    qc_report = {
        'slide_id': os.path.basename(path),
        'status': 'failed',
        'failure_reason': None
    }
    
    try:
        slide = openslide.OpenSlide(path)
        metadata = extract_slide_metadata(slide)
        
        print(f"  📋 Vendor: {metadata.get('vendor', 'unknown')}")
        print(f"  📏 Size: {metadata['width']}×{metadata['height']}")
        
        tiles, qc_stats = extract_qc_tiles(slide, max_tiles=NUM_CNN_TILES, blur_threshold=blur_threshold)
        slide.close()
        
        if len(tiles) < MIN_TILES_PER_SLIDE:
            qc_report['failure_reason'] = f"Insufficient tiles: {len(tiles)} < {MIN_TILES_PER_SLIDE}"
            qc_report['num_tiles'] = len(tiles)
            return None, qc_report
        
        features, feature_stats = extractor.extract(tiles)
        
        if features is None:
            qc_report['failure_reason'] = "Feature extraction failed"
            return None, qc_report
        
        # Flatten features
        flat = {}
        for k, v in features.items():
            for i, val in enumerate(v):
                flat[f"{k}_{i}"] = float(val)
        
        # Add metadata
        flat.update({
            'slide_id': os.path.basename(path),
            'num_tiles': len(tiles),
            'slide_width': metadata['width'],
            'slide_height': metadata['height'],
            'vendor': metadata['vendor'],
            'objective_power': metadata['objective_power'],
            'adaptive_blur_threshold': blur_threshold
        })
        
        # Update QC report
        qc_report.update({
            'status': 'success',
            'num_tiles': len(tiles),
            'qc_sampled': qc_stats.get('total_sampled', 0),
            'qc_passed': qc_stats.get('passed_tissue', 0),
            'qc_blur_failed': qc_stats.get('failed_blur', 0),
            'qc_artifact_failed': qc_stats.get('failed_artifacts', 0),
            'feature_dim': feature_stats['feature_dim'],
            'outliers_removed': feature_stats['num_outliers_removed'],
            'adaptive_blur_threshold': blur_threshold
        })
        
        return flat, qc_report
        
    except Exception as e:
        qc_report['failure_reason'] = str(e)
        print(f"  ✗ Processing error: {e}")
        return None, qc_report

# ===============================
# MAIN PIPELINE
# ===============================

def main():
    print("\n" + "="*70)
    
    # Get all files
    all_files = [f for f in os.listdir(SVS_DIR) if f.lower().endswith(".svs")]
    
    if len(all_files) == 0:
        print("❌ No SVS files found!")
        return
    
    # STEP 1: CALIBRATION (skip if blur filtering disabled)
    if USE_ADAPTIVE_BLUR and not GLOBAL_QC_STATS['calibration_complete']:
        # Use first 10 slides for calibration
        calibration_slides = [os.path.join(SVS_DIR, f) for f in all_files[:min(10, len(all_files))]]
        adaptive_blur_threshold = calibrate_blur_threshold(calibration_slides)
    else:
        adaptive_blur_threshold = FALLBACK_BLUR_THRESHOLD
        if not USE_ADAPTIVE_BLUR:
            print(f"ℹ️  Blur filtering disabled - uniform slide quality assumed")
            print(f"   (Typical for same-scanner, same-protocol datasets)\n")
    
    # STEP 2: PROCESSING
    extractor = DinoViTExtractor()
    checkpoint = load_checkpoint()
    processed = set(checkpoint["processed"])
    failed = set(checkpoint.get("failed", []))
    
    remaining = [f for f in all_files if f not in processed and f not in failed]
    
    print(f"\n📊 Pipeline Status:")
    print(f"   Total slides: {len(all_files)}")
    print(f"   Completed: {len(processed)}")
    print(f"   Failed: {len(failed)}")
    print(f"   Remaining: {len(remaining)}\n")
    
    if len(remaining) == 0:
        print("✅ All slides processed!\n")
        return
    
    results = []
    qc_reports = []
    newly_failed = []
    
    for idx, fname in enumerate(remaining, 1):
        print(f"\n{'='*70}")
        print(f"[{idx}/{len(remaining)}] Processing: {fname}")
        print(f"{'='*70}")
        log_progress(f"Starting: {fname}")
        
        path = os.path.join(SVS_DIR, fname)
        
        feats, qc_report = process_slide_q1(path, extractor, adaptive_blur_threshold)
        qc_reports.append(qc_report)
        
        if feats is None:
            print(f"  ❌ FAILED: {qc_report['failure_reason']}")
            log_progress(f"Failed: {fname} - {qc_report['failure_reason']}")
            newly_failed.append(fname)
            continue
        
        results.append(feats)
        processed.add(fname)
        
        print(f"  ✅ SUCCESS")
        log_progress(f"Completed: {fname}")
        
        # Periodic save
        if len(results) % SAVE_FREQUENCY == 0:
            print(f"\n💾 Checkpoint save ({len(results)} slides)...")
            pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
            pd.DataFrame(qc_reports).to_csv(QC_REPORT_CSV, index=False)
            save_checkpoint(list(processed), list(failed) + newly_failed)
    
    # Final save
    if len(results) > 0:
        print(f"\n{'='*70}")
        print("💾 FINAL SAVE")
        print(f"{'='*70}")
        pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
        pd.DataFrame(qc_reports).to_csv(QC_REPORT_CSV, index=False)
        save_checkpoint(list(processed), list(failed) + newly_failed)
        
        print(f"✅ Features: {OUTPUT_CSV}")
        print(f"✅ QC Report: {QC_REPORT_CSV}")
        print(f"✅ QC Stats: {QC_STATISTICS_JSON}")
        print(f"✅ Processed: {len(results)} slides")
        print(f"❌ Failed: {len(newly_failed)} slides\n")
    
    print("="*70)
    print("PIPELINE COMPLETED")
    print("="*70 + "\n")

if __name__ == "__main__":
    main()

Q1-READY HISTOLOGY PIPELINE - OPTIMIZED FOR LOW-CONTRAST DATA
🔧 Device: cpu
🔬 Model: DINO-ViT
📊 Adaptive Blur: DISABLED (uniform quality)
🎯 Blur Percentile: 5th
📁 Input: C:\Users\Shahinur\Downloads\PKG_Dataset\PKG - Brain-Mets-Lung-MRI-Path-Segs_histopathology images\data


ℹ️  Blur filtering disabled - uniform slide quality assumed
   (Typical for same-scanner, same-protocol datasets)

🔄 Loading DINO-ViT model...
✅ DINO-ViT loaded successfully


📊 Pipeline Status:
   Total slides: 111
   Completed: 0
   Failed: 0
   Remaining: 111


[1/111] Processing: YG_0CBM148C1MFN_wsi.svs
  📋 Vendor: aperio
  📏 Size: 22167×51785
  📐 Level 0, Downsample: 1.00×
  📏 Dimensions: 22167×51785
  🎯 Blur Threshold: 0.10
  ✅ QC Summary:
     Sampled: 1597
     Passed tissue: 100
     Failed blur: 0
     Failed artifacts: 0
     Failed stain: 0
     Final tiles: 100
     Blur (passed): 0.42 ± 0.11
  🔬 Extracting DINO features from 100 tiles...
  ✅ Feature shape: (100, 384)
  🔍 Removed 37 outlier tiles
  ✅ SU